In [ ]:
#UTF-8 (Unicode Transformation Format-8)

In [ ]:
print ("""TFM: RISK MANAGEMENT IN IBERIAN SPOT MARKET.""")   

In [ ]:
"""
01. DATA AQUISITION. 
(Brief description of API requests and Spot Market basics.)
"""
import pandas as pd
import requests # When you ping a website or portal for information this is called making a request.
import json # JavaScript Object Notation. Storing and Exanging files. 
import os # You can find important information about your location or about the process. 
import win32com.client as win32
import datetime

# Import numpy as np and import matploltlib(plt as pyplot)
%pylab inline 

#Imports esios library. You can find data quisition,clear and preparation functions. Please read before continuing this notebook.
import esios 

os.getcwd()  #Current working directory.

In [ ]:
# Working directory. esios.py must be there. Introduce your own working directory.
os.chdir('C:\\Users\\user\\Desktop\\fwdvamoscoo')

In [ ]:
# Header API request. In esios.py package introduce your own token.- consultasios@ree.es
esios.header() 

In [ ]:
#Dictionary with json files.
esios.archives_json_dict() 

In [ ]:
#Dictionary with json indicators. Data core of this project.
esios_indicators = esios.indicators_dict()
esios_indicators

In [ ]:
# lengh of esios_indicators library or Number of indicators.
len(esios_indicators)

In [ ]:
#Sort esios_indicators dictionary by key, giving a list. Introducing a 'x[0]' will sort by value, alphabetically.

#(lambda operator can have any number of arguments, but it can have only one expression.
#It cannot contain any statements and it returns a function object which can be assigned to any variable.)

sorted_esios_indicators = sorted(esios_indicators.items(), key=lambda x: x[1])
sorted_esios_indicators

In [ ]:
#Dataframe wich contains all agents present in  Iberian Spot market.
programunits = esios.PUs() 
programunits

In [ ]:
#Short description of programunits dataframe.
programunits.describe() 

In [ ]:
#Info on 'programmunits'.
programunits.info()

In [ ]:
# This funtion returns a list with coincidences.
esios.coincidences('solar')

In [ ]:
#Alternative way to download "UNIDADES DE PROGRAMACION" dataset.
#path ="C:............/Data.csv" #Download "UNIDADES DE PORGRAMACION" from www.esios.ree.es, rename as Data.
#programunits = pd.read_csv(path, sep=';')
#programunits.head()

In [ ]:
#Columns tittles of 'programunits'
programunits.columns.values.tolist()


In [ ]:
#Erase spaces from dataframe head.
programunits.columns = programunits.columns.str.replace(' ','')
programunits

In [ ]:
#Is there any NAN in programunits?
programunits.isnull().values.any()

In [ ]:
#Where are this NaN, if they were.
programunits.isnull().sum()

In [ ]:
#DISPLAY dataframe cointaining NAN.
programunitsNAN = programunits[programunits.isnull().any(axis=1)]
programunitsNAN

In [ ]:
#Change NAN for Porfolio, if they were.
programunits.fillna('PORFOLIO', inplace=True)
programunits

In [ ]:
#Unique values in column 'ProductionType', All kinds of power generation.
programunits.ProductionType.unique()

In [ ]:
#This previous propaedeutic can help and give a big picture for those not familiarized with Iberian Power Market.

In [ ]:
"""
02. DATA CLEANING AND PREPARATION.
"""

In [ ]:
# Indicators chosen for this project.

#Please introduce starting and finishing dates of the period uou want to study in this format "YYYYMMDD" . 
# You can add or substract inputs.
#Batch() returns Train and Test DataSet

ini, fin = '20190102', '20190111'
indList =[541,10249, 460,600,39,74,82, 10027,10034,84,85,10010, 762]

TrainD =esios.Batch(ini , fin, indList)

#except 762 'Net Balance' in EvalD
EvalD = esios.Batch(ini,fin, indList[:-1])

#TrainD

In [ ]:
# Data selection and preparation.

from sklearn.model_selection import train_test_split


X = TrainD[['541','10249','10027','39','82','74','460','10010','84','85','10034']]
X = X.fillna(0)
X['hour'] = [x.hour for x in X.index]
X['weekday'] = [x.weekday() for x in X.index]
X['weekend'] = [1 if x>=5 else 0 for x in X['weekday']]
X['dif_demp48'] = X['10027']-X['460']
X['dif_eol48'] = X['10010']-X['541']
X['windpower/Load'] = X['541'] / X['460'] 
X['P48SOLAR']= X['84']+X['85']
X['dif_SOLP48'] = X['P48SOLAR']-X['10034']
X['Residual_Load/Load'] = X['10249'] / X['460'] 

y = TrainD[['762']]#762

df_corr = pd.merge(X, y, left_index=True, right_index=True)
correl = df_corr.corr()
correl[['762']].sort_values(by='762', ascending=False)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)

In [ ]:
#Imput correlation heatmap. 

import seaborn as sns
%pylab inline

sns.heatmap(TrainD.corr(),annot = True)

In [ ]:

#RANDOM FOREST

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score


regRF = RandomForestRegressor(n_estimators= 5000)#5000 trees
regRF.fit(X_train, y_train.values.ravel())
predTest = regRF.predict(X_test)

r2_test = r2_score(y_test, predTest)
r2_test


In [ ]:
#Inputs significance.

signi = dict(zip(X.columns, regRF.feature_importances_)) 
signi

In [ ]:
#GRADIENT BOOSTING

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error

df_corr = pd.merge(X, y, left_index=True, right_index=True)
correl = df_corr.corr()
correl[['762']].sort_values(by='762', ascending=False)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)

pin = {'learning_rate': 0.1, 'max_depth': 5, 'subsample': 0.8}
gbr = GradientBoostingRegressor(n_estimators=5000, **pin)

gbr.fit(X_train, y_train.values.ravel())                       
predTestG = gbr.predict(X_test)
                       
r2_test = r2_score(y_test, predTestG)
r2_test

In [ ]:
#K-NEIGHBORS

from sklearn.neighbors import KNeighborsRegressor        

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)

kn=3
knn = KNeighborsRegressor(n_neighbors = kn)
knn.fit(X_train, y_train.values.ravel())
testPred = knn.predict(X_test)
r2_test = r2_score(y_test, testPred)
knnPred = knn.predict(X_test)

r2_test = r2_score(y_test, knnPred)
r2_test
        
    

In [ ]:
'''
04. VISUALIZATION.
'''

In [ ]:
# Wind Power Generation vs Load Graphic.
plt.plot( X['windpower/Load'], marker='.', color='blue', linewidth=0.25, alpha=0.5)
plt.title("Wind Power vs Load")
plt.xlabel("Date")
plt.ylabel("MW/Load")

In [ ]:
#Visualization of solar P48 program.
plt.margins(xmargin=0.000000015)
plt.plot( X['P48SOLAR'], marker='.', color='orange', linewidth=0.15, alpha=0.5)
plt.title("Solar p48 program")
plt.xlabel("Date")
plt.ylabel("MW")

In [ ]:
#Visualization of output parameter, Net Balance in Iberian Grid.
plt.plot( TrainD[['762']], marker='.', color='green', linewidth=0.5, alpha=0.5)
plt.title("Balancegrid")
plt.xlabel("Date")
plt.ylabel("MW")

In [ ]:
#Plotting Evaluation data
EvalD.plot(figsize=(15,10), title= 'Daily values', fontsize=20)
plt.show()